In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/HiWi/Aircraft_Noise

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/HiWi/Aircraft_Noise


In [ ]:
! pip install  typing traffic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [ ]:
! pip install pandas joblib openpyxl matplotlib



In [ ]:
!pip install folium


In [ ]:
!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.6 MB/s eta 0:00:00


# Loading ADS-B Data

In [ ]:
import joblib
import pandas as pd

# Load ADS-B Data (Trajectory Data)
adsb_data = joblib.load("data_2022_january.joblib")

df_adsb = adsb_data.data

In [ ]:
print("ADS-B Data Loaded:")
display(df_adsb.info())

ADS-B Data Loaded:
<class 'pandas.core.frame.DataFrame'>
Index: 13897138 entries, 0 to 12381
Data columns (total 21 columns):
 #   Column         Dtype              
---  ------         -----              
 0   timestamp      datetime64[ns, UTC]
 1   icao24         object             
 2   latitude       float64            
 3   longitude      float64            
 4   groundspeed    float64            
 5   track          float64            
 6   vertical_rate  float64            
 7   callsign       object             
 8   onground       bool               
 9   alert          bool               
 10  spi            bool               
 11  squawk         object             
 12  altitude       float64            
 13  geoaltitude    float64            
 14  last_position  datetime64[ns, UTC]
 15  hour           datetime64[ns, UTC]
 16  firstseen      datetime64[ns, UTC]
 17  origin         object             
 18  lastseen       datetime64[ns, UTC]
 19  destination    object        

None

In [ ]:
display(df_adsb.head())

,timestamp,icao24,latitude,longitude,groundspeed,track,vertical_rate,callsign,onground,alert,...,squawk,altitude,geoaltitude,last_position,hour,firstseen,origin,lastseen,destination,day
0,2021-12-31 22:00:01+00:00,3e0fdf,52.845635,8.701250,90.0,303.157924,384.0,CHX86,False,False,...,20,1150.0,1450.0,2021-12-31 21:59:58.720999936+00:00,2021-12-31 22:00:00+00:00,2021-12-31 21:33:03+00:00,EDDV,2021-12-31 22:54:09+00:00,EDWR,2021-12-31 00:00:00+00:00
1,2021-12-31 22:00:02+00:00,3e0fdf,52.846333,8.699524,90.0,303.157924,384.0,CHX86,False,False,...,20,1200.0,1500.0,2021-12-31 22:00:01.660000+00:00,2021-12-31 22:00:00+00:00,2021-12-31 21:33:03+00:00,EDDV,2021-12-31 22:54:09+00:00,EDWR,2021-12-31 00:00:00+00:00
2,2021-12-31 22:00:03+00:00,3e0fdf,52.846573,8.698959,90.0,303.157924,384.0,CHX86,False,False,...,20,1200.0,1500.0,2021-12-31 22:00:02.780000+00:00,2021-12-31 22:00:00+00:00,2021-12-31 21:33:03+00:00,EDDV,2021-12-31 22:54:09+00:00,EDWR,2021-12-31 00:00:00+00:00
3,2021-12-31 22:00:04+00:00,3e0fdf,52.846799,8.698268,90.0,303.157924,320.0,CHX86,False,False,...,20,1200.0,1500.0,2021-12-31 22:00:03.816000+00:00,2021-12-31 22:00:00+00:00,2021-12-31 21:33:03+00:00,EDDV,2021-12-31 22:54:09+00:00,EDWR,2021-12-31 00:00:00+00:00
4,2021-12-31 22:00:05+00:00,3e0fdf,52.847031,8.697719,90.0,303.157924,192.0,CHX86,False,False,...,20,1175.0,1475.0,2021-12-31 22:00:04.828000+00:00,2021-12-31 22:00:00+00:00,2021-12-31 21:33:03+00:00,EDDV,2021-12-31 22:54:09+00:00,EDWR,2021-12-31 00:00:00+00:00


# Export processed ADS-B data to CSV

In [ ]:
df_adsb.to_csv("processed_adsb_data.csv", index=False)
print("ADS-B data exported to 'processed_adsb_data.csv'")

from google.colab import files
files.download("processed_adsb_data.csv")


In [ ]:
# Extract the date part (YYYY-MM-DD) for grouping
df_adsb["date"] = df_adsb["timestamp"].dt.date

# Sort by time (to maintain trajectory order)
df_adsb = df_adsb.sort_values(by=["icao24", "date", "timestamp"])

# Display processed data
display(df_adsb.head())

,timestamp,icao24,latitude,longitude,groundspeed,track,vertical_rate,callsign,onground,alert,...,altitude,geoaltitude,last_position,hour,firstseen,origin,lastseen,destination,day,date
10969,2022-01-12 21:02:25+00:00,01022b,34.918579,27.504497,438.0,326.926773,0.0,MSC2932,False,False,...,35975.0,35900.0,2022-01-12 21:02:24.894000128+00:00,2022-01-12 21:00:00+00:00,2022-01-12 21:02:22+00:00,NaN,2022-01-13 00:12:45+00:00,EDDV,2022-01-13 00:00:00+00:00,2022-01-12
10970,2022-01-12 21:02:26+00:00,01022b,34.920090,27.503264,438.0,326.926773,0.0,MSC2932,False,False,...,35975.0,35900.0,2022-01-12 21:02:25.644000+00:00,2022-01-12 21:00:00+00:00,2022-01-12 21:02:22+00:00,NaN,2022-01-13 00:12:45+00:00,EDDV,2022-01-13 00:00:00+00:00,2022-01-12
10971,2022-01-12 21:02:27+00:00,01022b,34.920090,27.503264,438.0,326.926773,0.0,MSC2932,False,False,...,35975.0,35900.0,2022-01-12 21:02:25.644000+00:00,2022-01-12 21:00:00+00:00,2022-01-12 21:02:22+00:00,NaN,2022-01-13 00:12:45+00:00,EDDV,2022-01-13 00:00:00+00:00,2022-01-12
10972,2022-01-12 21:02:28+00:00,01022b,34.923477,27.500573,438.0,326.926773,0.0,MSC2932,False,False,...,35975.0,35900.0,2022-01-12 21:02:27.624999936+00:00,2022-01-12 21:00:00+00:00,2022-01-12 21:02:22+00:00,NaN,2022-01-13 00:12:45+00:00,EDDV,2022-01-13 00:00:00+00:00,2022-01-12
10973,2022-01-12 21:02:29+00:00,01022b,34.924347,27.499900,439.0,326.817282,0.0,MSC2932,False,False,...,35975.0,35900.0,2022-01-12 21:02:28.332999936+00:00,2022-01-12 21:00:00+00:00,2022-01-12 21:02:22+00:00,NaN,2022-01-13 00:12:45+00:00,EDDV,2022-01-13 00:00:00+00:00,2022-01-12


# Grouped flight summary

In [ ]:
# Sort by time (to maintain trajectory order)
df_adsb.groupby(["icao24","day"]).timestamp.min()

In [ ]:
# Create a unique flight identifier by grouping by (icao24, date)
df_adsb["flight_id"] = df_adsb.groupby(["icao24", "date"]).ngroup()

In [ ]:
grouped_data = df_adsb.groupby("flight_id").agg({
    "icao24": "first",
    "callsign": "first",
    "date": "first",
    "timestamp": ["min", "max"]
}).reset_index()

grouped_data.columns = ["flight_id", "icao24", "callsign", "date", "first_seen", "last_seen"]

print("Grouped flight data:")
display(grouped_data.head())


Grouped flight data:


,flight_id,icao24,callsign,date,first_seen,last_seen
0,0,01022b,MSC2932,2022-01-12,2022-01-12 21:02:25+00:00,2022-01-12 23:59:59+00:00
1,1,01022b,MSC2932,2022-01-13,2022-01-13 00:00:01+00:00,2022-01-13 05:27:37+00:00
2,2,01022d,MSC2932,2022-01-05,2022-01-05 21:14:34+00:00,2022-01-05 23:59:59+00:00
3,3,01022d,MSC2932,2022-01-06,2022-01-06 00:00:01+00:00,2022-01-06 05:35:29+00:00
4,4,01022d,MSC2932,2022-01-19,2022-01-19 20:24:28+00:00,2022-01-19 23:59:59+00:00


# Getting Trajectories

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import folium

# --- Dropdown Widgets ---
icao24_list = sorted(df_adsb["icao24"].dropna().unique())

icao24_dropdown = widgets.Dropdown(
    options=icao24_list,
    description="Select ICAO24:",
    layout=widgets.Layout(width='50%')
)

timestamp_dropdown = widgets.Dropdown(
    options=[],
    description="Select Date:",
    layout=widgets.Layout(width='50%')
)

plot_type_dropdown = widgets.Dropdown(
    options=["Simple Graph", "Folium Map", "Both"],
    description="Plot Type:",
    layout=widgets.Layout(width='50%')
)

output = widgets.Output()

# --- Helper Functions ---
def get_flight_id(icao24, date):
    result = df_adsb[(df_adsb["icao24"] == icao24) & (df_adsb["date"] == date)]
    if result.empty:
        return None
    return result["flight_id"].unique()[0]  # One flight per icao24+date assumed

def plot_matplotlib_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    plt.figure(figsize=(10, 6))
    plt.plot(flight_data["longitude"], flight_data["latitude"], marker="o", linestyle="-", markersize=4,color="#FF10F0")
    title = (
        f"Flight ID: {flight_id} | ICAO24: {icao24} | Callsign: {callsign}\n"
        f"First Seen: {firstseen} | Last Seen: {lastseen}"
    )
    plt.title(title, fontsize=11)
    plt.annotate("Start", (flight_data["longitude"].iloc[0], flight_data["latitude"].iloc[0]), fontsize=9, color='green')
    plt.annotate("End", (flight_data["longitude"].iloc[-1], flight_data["latitude"].iloc[-1]), fontsize=9, color='red')
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_folium_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    start_coords = [flight_data["latitude"].iloc[0], flight_data["longitude"].iloc[0]]
    m = folium.Map(location=start_coords, zoom_start=7)
    coords = list(zip(flight_data["latitude"], flight_data["longitude"]))
    folium.PolyLine(coords, color='#FF10F0', weight=3, opacity=0.8).add_to(m)
    folium.Marker(coords[0], popup="Start", icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(coords[-1], popup="End", icon=folium.Icon(color='red')).add_to(m)

    folium.map.Marker(
        start_coords,
        icon=folium.DivIcon(
            html=f"""
            <div style="font-size: 12pt">
                <b>Flight ID:</b> {flight_id}<br>
                <b>ICAO24:</b> {icao24}<br>
                <b>Callsign:</b> {callsign}<br>
                <b>First Seen:</b> {firstseen}<br>
                <b>Last Seen:</b> {lastseen}
            </div>
            """
        )
    ).add_to(m)

    return m

# --- Widget Event Handlers ---
def on_icao24_change(change):
    icao24 = change['new']
    matching_dates = df_adsb[df_adsb["icao24"] == icao24]["date"].dropna().unique()
    timestamp_dropdown.options = sorted(matching_dates)

def on_plot_button_clicked(b):
    with output:
        clear_output()
        icao24 = icao24_dropdown.value
        date = timestamp_dropdown.value
        plot_type = plot_type_dropdown.value
        flight_id = get_flight_id(icao24, date)

        if flight_id is None:
            print("No flight data found for the selection.")
            return

        flight_data = df_adsb[df_adsb["flight_id"] == flight_id]

        if plot_type == "Simple Graph":
            plot_matplotlib_trajectory(flight_data, flight_id)
        elif plot_type == "Folium Map":
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)
        elif plot_type == "Both":
            plot_matplotlib_trajectory(flight_data, flight_id)
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)

# --- Plot Button ---
plot_button = widgets.Button(description="Plot Flight", button_style='primary')

# Attach handlers
icao24_dropdown.observe(on_icao24_change, names='value')
plot_button.on_click(on_plot_button_clicked)

# Initial trigger
on_icao24_change({'new': icao24_dropdown.value})

# --- Display UI ---
display(icao24_dropdown, timestamp_dropdown, plot_type_dropdown, plot_button, output)


Dropdown(description='Select ICAO24:', layout=Layout(width='50%'), options=('01022b', '01022d', '01022e', '010…

Dropdown(description='Select Date:', layout=Layout(width='50%'), options=(datetime.date(2022, 1, 12), datetime…

Dropdown(description='Plot Type:', layout=Layout(width='50%'), options=('Simple Graph', 'Folium Map', 'Both'),…

Button(button_style='primary', description='Plot Flight', style=ButtonStyle())

Output()

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import folium
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting

# --- Dropdown Widgets ---
icao24_list = sorted(df_adsb["icao24"].dropna().unique())

icao24_dropdown = widgets.Dropdown(
    options=icao24_list,
    description="Select ICAO24:",
    layout=widgets.Layout(width='50%')
)

timestamp_dropdown = widgets.Dropdown(
    options=[],
    description="Select Date:",
    layout=widgets.Layout(width='50%')
)

plot_type_dropdown = widgets.Dropdown(
    options=["Simple Graph", "Folium Map", "3D Plot", "Both", "All (2D + 3D + Map)"],
    description="Plot Type:",
    layout=widgets.Layout(width='50%')
)

output = widgets.Output()

# --- Helper Functions ---
def get_flight_id(icao24, date):
    result = df_adsb[(df_adsb["icao24"] == icao24) & (df_adsb["date"] == date)]
    if result.empty:
        return None
    return result["flight_id"].unique()[0]

def plot_matplotlib_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    plt.figure(figsize=(10, 6))
    plt.plot(flight_data["longitude"], flight_data["latitude"], marker="o", linestyle="-", markersize=4, color="#FF10F0")
    title = (
        f"Flight ID: {flight_id} | ICAO24: {icao24} | Callsign: {callsign}\n"
        f"First Seen: {firstseen} | Last Seen: {lastseen}"
    )
    plt.title(title, fontsize=11)
    plt.annotate("Start", (flight_data["longitude"].iloc[0], flight_data["latitude"].iloc[0]), fontsize=9, color='green')
    plt.annotate("End", (flight_data["longitude"].iloc[-1], flight_data["latitude"].iloc[-1]), fontsize=9, color='red')

    box_text = (
        f"Flight ID: {flight_id}\n"
        f"ICAO24: {icao24}\n"
        f"Callsign: {callsign}\n"
        f"First Seen: {firstseen}\n"
        f"Last Seen: {lastseen}"
    )
    plt.gca().text(
        0.02, 0.98, box_text,
        transform=plt.gca().transAxes,
        fontsize=10,
        verticalalignment='top',
        bbox=dict(boxstyle="round,pad=0.5", facecolor="white", edgecolor="#FF10F0", alpha=0.8)
    )

    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_3d_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(
        flight_data["longitude"],
        flight_data["latitude"],
        flight_data["altitude"],
        color="#FF10F0",
        linewidth=2,
        marker='o',
        markersize=3
    )

    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_zlabel("Altitude (m)")
    ax.set_title("3D Flight Trajectory", fontsize=14, pad=20)

    box_text = (
        f"Flight ID: {flight_id}\n"
        f"ICAO24: {icao24} | Callsign: {callsign}\n"
        f"First Seen: {firstseen}\n"
        f"Last Seen: {lastseen}"
    )
    ax.text2D(0.02, 0.95, box_text, transform=ax.transAxes,
              bbox=dict(boxstyle="round,pad=0.5", facecolor="white", edgecolor="#FF10F0", alpha=0.9),
              fontsize=10)
    plt.tight_layout()
    plt.show()

def plot_folium_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    start_coords = [flight_data["latitude"].iloc[0], flight_data["longitude"].iloc[0]]
    m = folium.Map(location=start_coords, zoom_start=7)
    coords = list(zip(flight_data["latitude"], flight_data["longitude"]))
    folium.PolyLine(coords, color='#FF10F0', weight=3, opacity=0.8).add_to(m)
    folium.Marker(coords[0], popup="Start", icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(coords[-1], popup="End", icon=folium.Icon(color='red')).add_to(m)

    info_html = f"""
    <div style="background-color:white; border:2px solid #FF10F0; border-radius:10px; padding:10px;">
        <b>Flight ID:</b> {flight_id}<br>
        <b>ICAO24:</b> {icao24}<br>
        <b>Callsign:</b> {callsign}<br>
        <b>First Seen:</b> {firstseen}<br>
        <b>Last Seen:</b> {lastseen}
    </div>
    """
    folium.Marker(
        location=start_coords,
        icon=folium.DivIcon(html=info_html)
    ).add_to(m)

    return m

# --- Widget Event Handlers ---
def on_icao24_change(change):
    icao24 = change['new']
    matching_dates = df_adsb[df_adsb["icao24"] == icao24]["date"].dropna().unique()
    timestamp_dropdown.options = sorted(matching_dates)

def on_plot_button_clicked(b):
    with output:
        clear_output()
        icao24 = icao24_dropdown.value
        date = timestamp_dropdown.value
        plot_type = plot_type_dropdown.value
        flight_id = get_flight_id(icao24, date)

        if flight_id is None:
            print("No flight data found for the selection.")
            return

        flight_data = df_adsb[df_adsb["flight_id"] == flight_id]

        if plot_type == "Simple Graph":
            plot_matplotlib_trajectory(flight_data, flight_id)
        elif plot_type == "Folium Map":
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)
        elif plot_type == "3D Plot":
            plot_3d_trajectory(flight_data, flight_id)
        elif plot_type == "Both":
            plot_matplotlib_trajectory(flight_data, flight_id)
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)
        elif plot_type == "All (2D + 3D + Map)":
            plot_matplotlib_trajectory(flight_data, flight_id)
            plot_3d_trajectory(flight_data, flight_id)
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)

# --- Plot Button ---
plot_button = widgets.Button(description="Plot Flight", button_style='primary')

# Attach handlers
icao24_dropdown.observe(on_icao24_change, names='value')
plot_button.on_click(on_plot_button_clicked)

# Initial trigger
on_icao24_change({'new': icao24_dropdown.value})

# --- Display UI ---
display(icao24_dropdown, timestamp_dropdown, plot_type_dropdown, plot_button, output)


Dropdown(description='Select ICAO24:', layout=Layout(width='50%'), options=('01022b', '01022d', '01022e', '010…

Dropdown(description='Select Date:', layout=Layout(width='50%'), options=(datetime.date(2022, 1, 12), datetime…

Dropdown(description='Plot Type:', layout=Layout(width='50%'), options=('Simple Graph', 'Folium Map', '3D Plot…

Button(button_style='primary', description='Plot Flight', style=ButtonStyle())

Output()